In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:

masterfinal = pd.read_csv("Masterlist of Schools.csv")
location = pd.read_csv("Schools Location Data.csv")
rooms = pd.read_csv("Rooms data.csv")
mooe = pd.read_csv("MOOE data.csv")
teachers = pd.read_csv("Teachers data.csv")
lit = pd.read_csv("Regional_Functional_Literacy_Rate_2015.csv")
stratio = pd.read_csv("stratio.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Masterlist of Schools.csv'

In [ ]:
masterfinal['school.cityincome'].unique()


In [ ]:
#mean number of teachers per city and compare that to income of that city 
#aggregation of data per city in terms of the number of teachers and correlate it with the income of data 
#generate scatter plot - see if theres a negative or positive correlation 

In [ ]:
city_income_corrections={
    "P 25 M or more but less than P 35 M":30000000,
    "P 35 M or more but less than P 45 M":40000000,
    "P 15 M or more but less than P 25 M":20000000,
    "P 45 M or more but less than P 55 M":50000000,
    "P 160 M or more but less than P 240 M":200000000,
    "P 240 M or more but less than P 320 M":280000000,
    "P 55 M or more":55000000,
    "P 320 M or more but less than P 400 M":360000000,
    "P 400 M or more":400000000,
    "Below P 15 M":15000000,
    "P 80 M or more but less than P 160 M":120000000
}

incorrect_city_income_format = list(city_income_corrections.keys())

def assign_city_income(x):
  if x["school.cityincome"] in incorrect_city_income_format:
    corrected_city_income=city_income_corrections[x["school.cityincome"]]
    x["school.cityincome"]=corrected_city_income

  return x

masterfinal = masterfinal.apply(lambda x: assign_city_income(x), axis=1)


masterfinal.dropna()
masterfinal = masterfinal[masterfinal["school.cityincome"] != "Special Class"]

masterfinal

In [ ]:
df_all = pd.merge(masterfinal, teachers, how='left', on ='school.id')
df_all.groupby(["school.province"])[["school.cityincome", "teachers.instructor", "teachers.mobile", "teachers.regular", "teachers.sped"]].sum()

df_all["total_teachers"] = (df_all["teachers.instructor"] +df_all["teachers.mobile"]+df_all["teachers.regular"]+df_all["teachers.sped"])



In [ ]:

df_enrollment = pd.merge(masterfinal, mooe, how = 'left', on = 'school.id')


df_enrollment_final = df_enrollment.groupby(["school.province"])[["school.cityincome", "school.enrollment"]].apply(lambda x : x.sum())
df_enrollment_final
#df_enrollment_final.columns = ['Schools_Income', 'Schools_Enrollment']



In [ ]:
df_enrollment_final

In [ ]:
lit

In [ ]:
stratio

In [ ]:
df_rooms = pd.merge(masterfinal, rooms, how='left', on ='school.id')
df_rooms.groupby(["school.province"])[["school.cityincome", "rooms.standard.academic", "rooms.standard.unused", "rooms.nonstandard.academic", "rooms.nonstandard.unused"]].sum()

df_rooms["total_rooms"] = (df_rooms["rooms.standard.academic"] +df_rooms["rooms.standard.unused"]+df_rooms["rooms.nonstandard.academic"]+df_rooms["rooms.nonstandard.unused"])

df_rooms_final = df_rooms.groupby(["school.province"])[["school.cityincome", "total_rooms"]].apply(lambda x : x.sum())

df_rooms_final.columns = ['Schools_Income', 'Schools_Rooms']
df_rooms_final


In [ ]:
# Get indexes where name column doesn't have value 
indexNames = df_all[(df_all['school.cityincome'] == 'Special Class')].index 
# Delete these row indexes from dataFrame
df_all.drop(indexNames, inplace=True)



df_final = df_all.groupby(["school.province"])[["school.cityincome", "total_teachers"]].apply(lambda x : x.sum())
df_final.columns = ['Schools_Income', 'Schools_Teachers']

df_final



In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_final["Schools_Income"], df_final["Schools_Teachers"])
plt.title("Relationship Between Income Level of a Given Region and Teacher Availability", fontsize=14)
plt.ylabel("Number of Teachers")
plt.xlabel("Income Level (1*10^11)")
plt.show()
#df_final["ratio"] = df_final["total_teachers"]/df_final["school.cityincome"]

#df_final.plot.bar(stacked=True)




In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_enrollment_final["Schools_Income"], df_enrollment_final["Schools_Enrollment"])
plt.title("Relationship Between Income Level of a Given Region and Enrollment", fontsize=14)
plt.ylabel("Number of Enrollment")
plt.xlabel("Income Level (1*10^11)")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_rooms_final["Schools_Income"], df_rooms_final["Schools_Rooms"])
plt.title("Relationship Between Income Level of a Given Region and Room Availability", fontsize=14)
plt.ylabel("Number of Rooms Available")
plt.xlabel("Income Level (1*10^11)")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(lit["school.cityincome"], lit["literacy_rate"])
plt.title("Relationship Between Income Level of a Given Region and Literacy Rate", fontsize=14)
plt.ylabel("Literacy Rate")
plt.xlabel("Income Level (1*10^11)")
plt.show()
lit.corr()


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(stratio["school.cityincome"], stratio["student_teacher_ratio"])
plt.title("Relationship Between Income Level of a Given Region and Student Teacher Ratio", fontsize=14)
plt.ylabel("Student Teacher Ratio")
plt.xlabel("Income Level (1*10^11)")
plt.show()
stratio.corr()

In [ ]:
merged_df = pd.merge(df_enrollment_final, df_rooms_final, on="Schools_Income")
merged_df.columns = ['Schools_Income', 'Schools_Enrollment', 'Schools_Rooms']


#df_enrollment_final.columns = ['Schools_Income', 'Schools_Enrollment']

merged_df_2 = pd.merge(merged_df, df_final, on="Schools_Income")
merged_df_2.columns = ['Schools_Income', 'Schools_Enrollment', 'Schools_Rooms', 'Schools_Teachers']

merged_df_2

merged_df_2.corr()

In [ ]:
plt.figure(figsize=(10,8))

sns.heatmap(merged_df_2.corr(), center=1, annot=True)

plt.show()

In [ ]:
cor1 = df_final.corr()
cor1

In [ ]:
cor2 = df_rooms_final.corr()
cor2

In [ ]:
cor3 = df_enrollment_final.corr()
cor3

In [8]:
df_heatmap = pd.concat([cor1, cor2, cor3], axis = 1, sort=False)
df_heatmap

NameError: name 'cor1' is not defined

In [ ]:
plt.figure(figsize=(10,8))

sns.heatmap(df_heatmap.corr(), center=1, annot=True)

plt.show()

In [ ]:
df_extra = pd.concat([df_final, teachers], axis = 0, sort=False)
df_extra_final = df_extra[["Schools_Income", "teachers.instructor", "teachers.mobile", "teachers.regular", "teachers.sped"]]
df_extra_final

In [ ]:
df_extra_final.corr()

In [ ]:
plt.figure(figsize=(10,8))

sns.heatmap(df_extra_final.corr(), center=0.00, annot=True)

plt.show()

In [ ]:
plt.figure(figsize=(10,8))

sns.heatmap(df_extra_final.corr(), center=1, annot=True)

plt.show()

In [ ]:
df_new = pd.concat([mooe, df_final], axis = 0, sort=False)
df_new.columns = ['School_ID', 'School_Name', 'School_Enrollment', "School_Offering", "School_MOOE", "School_Income", "School_Teachers"]

df_new

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_new["School_MOOE"], df_new["School_Enrollment"])
plt.title("Relationship Between Income Level of a Given Region and Teacher Availability", fontsize=14)
plt.ylabel("Number of Teachers")
plt.xlabel("Income Level (1*10^11)")
plt.show()
